---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [72]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [73]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [74]:
from collections import Counter
def degree_distribution(g):
    c = Counter(g.degree().values())
    d = {k:v/nx.number_of_nodes(g) for k, v in c.items()}
    return d

In [75]:
info = dict()
for i in range(5):
    g = P1_Graphs[i]
    temp = [len(degree_distribution(g)), 
            nx.average_clustering(g),
            nx.average_shortest_path_length(g)]
    info[i] = temp
info

{0: [29, 0.03167539146454044, 4.099161161161161],
 1: [7, 0.5642419635919628, 5.089871871871872],
 2: [5, 0.4018222222222227, 9.378702269692925],
 3: [37, 0.03780379975223251, 3.1048046283934134],
 4: [9, 0.0033037037037037037, 5.0785509568313305]}

In [76]:
def graph_identification():
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [77]:
G = nx.read_gpickle('email_prediction.txt')
# print(nx.info(G))

In [78]:
dep = [i[1]['Department'] for i in G.nodes(data=True)]
sal = [i[1]['ManagementSalary'] for i in G.nodes(data=True)]
df = pd.DataFrame({'dep': [i[1]['Department'] for i in G.nodes(data=True)], 
                   'sal': [i[1]['ManagementSalary'] for i in G.nodes(data=True)], 
                   'clu': pd.Series(nx.clustering(G)), 
                   'deg': pd.Series(G.degree()), 
                   'clo': pd.Series(nx.closeness_centrality(G, normalized=True)), 
                   'bet': pd.Series(nx.betweenness_centrality(G, normalized=True)), 
                   'pgr': pd.Series(nx.pagerank(G))})
df.head()

,bet,clo,clu,deg,dep,pgr,sal
0,0.001124,0.421991,0.276423,44,1,0.001224,0.0
1,0.001195,0.422360,0.265306,52,1,0.001426,NaN
2,0.006570,0.461490,0.297803,95,21,0.002605,NaN
3,0.001654,0.441663,0.384910,71,21,0.001833,1.0
4,0.005547,0.462152,0.318691,96,21,0.002526,1.0


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [81]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
def salary_predictions():
    df_train = df[~pd.isnull(df['sal'])]
    df_test = df[pd.isnull(df['sal'])]
    y_train = df_train['sal']
    x_train = df_train.drop('sal', axis=1)
    x_test = df_test.drop('sal', axis=1)
    clf = GradientBoostingClassifier(random_state=0, learning_rate = 0.1,max_depth = 6)
    clf.fit(x_train, y_train)
    y_predict = clf.predict_proba(x_test)[:,1]
    return pd.Series(y_predict, index=df_test.index)
salary_predictions()

1       0.000436
2       0.998698
5       0.998634
8       0.066826
14      0.003590
18      0.011567
27      0.017381
30      0.986600
31      0.064990
34      0.001809
37      0.001746
40      0.001437
45      0.000493
54      0.159930
55      0.902753
60      0.009169
62      0.998634
65      0.998555
77      0.000936
79      0.001520
97      0.001089
101     0.000358
103     0.679736
108     0.001131
113     0.003124
122     0.000143
141     0.009558
142     0.998634
144     0.001041
145     0.951941
          ...   
913     0.000978
914     0.000877
915     0.000193
918     0.002502
923     0.001418
926     0.003501
931     0.000589
934     0.000173
939     0.000165
944     0.000173
945     0.001418
947     0.002308
950     0.002930
951     0.001157
953     0.000209
959     0.000157
962     0.000156
963     0.002574
968     0.001675
969     0.003008
974     0.001430
984     0.000173
987     0.001533
989     0.002313
991     0.001583
992     0.000157
994     0.000173
996     0.0001

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [80]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
df2 = future_connections.copy()
df2['jaccard_coefficient'] = list(map(lambda x: list(nx.jaccard_coefficient(G, [x]))[0][2], df2.index))
df2['resource_allocation'] = list(map(lambda x: list(nx.resource_allocation_index(G, [x]))[0][2], df2.index))
df2['pref_attachment'] = list(map(lambda x: list(nx.preferential_attachment(G, [x]))[0][2], df2.index))
df2['community_neighbors'] = list(map(lambda x: list(nx.cn_soundarajan_hopcroft(G, [x], community='Department'))[0][2], df2.index))

In [ ]:
def new_connections_predictions():
    df_train = df2[~pd.isnull(df2['Future Connection'])]
    df_test = df2[pd.isnull(df2['Future Connection'])]
    y_train = df_train['Future Connection']
    x_train = df_train.drop('Future Connection', axis=1)
    x_test = df_test.drop('Future Connection', axis=1)
    clf = GradientBoostingClassifier(random_state=0, learning_rate = 0.1,max_depth = 4)
    clf.fit(x_train, y_train)
    y_predict = clf.predict_proba(x_test)[:,1]
    return pd.Series(y_predict, index=df_test.index)
new_connections_predictions()


(107, 348)    0.039009
(542, 751)    0.012901
(20, 426)     0.575559
(50, 989)     0.012901
(942, 986)    0.012901
(324, 857)    0.012901
(13, 710)     0.117178
(19, 271)     0.147201
(319, 878)    0.012901
(659, 707)    0.012901
(49, 843)     0.012901
(208, 893)    0.012901
(377, 469)    0.014036
(405, 999)    0.016506
(129, 740)    0.013091
(292, 618)    0.018084
(239, 689)    0.012901
(359, 373)    0.014734
(53, 523)     0.069195
(276, 984)    0.012901
(202, 997)    0.012901
(604, 619)    0.043061
(270, 911)    0.012901
(261, 481)    0.064915
(200, 450)    0.995465
(213, 634)    0.012901
(644, 735)    0.042362
(346, 553)    0.012901
(521, 738)    0.012901
(422, 953)    0.015818
                ...   
(672, 848)    0.012901
(28, 127)     0.980412
(202, 661)    0.012901
(54, 195)     0.998274
(295, 864)    0.012901
(814, 936)    0.012901
(839, 874)    0.012901
(139, 843)    0.012901
(461, 544)    0.013673
(68, 487)     0.013673
(622, 932)    0.012901
(504, 936)    0.017042
(479, 528) 